## Experimenting With Different Models

* 어떤 모델이 가장 좋은 성능을 보여줄까?
* scikit-learn [documentation](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html)에서 decision tree의 수 많은 옵션들을 볼 수 있음
* 대부분 가장 중요한 옵션들은 tree의 깊이(depth)를 결정함
    * tree의 깊이는 예측(가장 아래)에 도달하기까지 얼마나 많이 분할되었는지 나타냄
<img src="../images/2/ShallowTree.png" width="70%">

<br>
* tree가 깊어질수록 데이터셋은 더 적은 수로 나눠짐
    * tree가 1번 나눠지면(깊이는 1) 데이터셋은 2 그룹으로 나눠짐(split)
    * 여기서 한 번 더 나눠지면(깊이는 2) 데이터셋은 4 그룹으로 나눠짐
    * 각 레벨(깊이)에서 split이 더해지면 나눠진 데이터셋 그룹이 두배로 늘어남(각 그룹에 속하는 데이터 수는 줄어듦)
    * 깊이 10까지 데이터셋을 나누면 1024(2^10)개의 leaves를 얻을 수 있음
* 아주 적은 데이터를 가지고 있는 leaves는 실제 값에 가까운 예측을 할 수 있음 but 그만큼 새로운 데이터에 대해서는 예측을 잘 하지 못함(너무 적은 데이터를 기반으로 예측하기 때문)

<br>
* overfitting: 모델이 학습 데이터에 너무 fit돼서, 새로운 데이터나 validation data를 잘 예측하지 못하는 상황
* underfitting: 각 그룹에 속하는 데이터의 범위가 너무 넓어서, 예측을 잘 하지 못하는 상황(validation data뿐 아니라 학습 데이터도 잘 예측하지 못함)
    * 예를 들어, tree의 깊이가 1일 때(데이터가 단 2 그룹으로만 나누어졌을 때) 발생
<img src="../images/2/Under-Overfitting.png" width="90%">
* tree의 깊이가 깊어질수록 overfitting이 발생할 확률 증가
    * 한 그룹에 속하는 데이터가 적어져서 그 데이터들에 너무 꼭 들어맞게 학습되기 때문
    * 학습 데이터(training data)에 대한 error는 감소
    * 검증 데이터(validation data)에 대한 error는 증가
* tree의 깊이가 얕으면 underfitting이 발생할 확률 증가
    * 데이터의 범위가 너무 넓어 데이터의 중요한 특징이나 패턴을 모델이 잡아내지 못하기 때문
    * 학습 데이터와 검증 데이터 모두에서 error 증가

<br>

## Example

* *max_leaf_nodes*: overfitting과 underfitting을 조절할 수 있는 argument
* 위 그래프를 보면, tree의 깊어질수록 underfitting 영역에서 overfitting 영역으로 이동함

In [1]:
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor

def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    
    return mae

* get_mae(): *max_leaf_nodes*에 따라 달라지는 mae를 확인할 메소드

In [2]:
import pandas as pd
    
## Load data ##
melbourne_file_path = '../input/melb_data.csv'
melbourne_data = pd.read_csv(melbourne_file_path) 

## Filter rows with missing values ##
filtered_melbourne_data = melbourne_data.dropna(axis=0)

## Choose target and features ##
y = filtered_melbourne_data.Price
melbourne_features = ['Rooms', 'Bathroom', 'Landsize', 'BuildingArea', 
                        'YearBuilt', 'Lattitude', 'Longtitude']
X = filtered_melbourne_data[melbourne_features]

from sklearn.model_selection import train_test_split

## split data into training and validation data, for both features and target ##
train_X, val_X, train_y, val_y = train_test_split(X, y,random_state = 0)

In [3]:
## compare MAE with differing values of max_leaf_nodes ##
for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %d \t\t Mean Absolute Error: %d" %(max_leaf_nodes, my_mae))

Max leaf nodes: 5 		 Mean Absolute Error: 347380
Max leaf nodes: 50 		 Mean Absolute Error: 258171
Max leaf nodes: 500 		 Mean Absolute Error: 243495
Max leaf nodes: 5000 		 Mean Absolute Error: 254983
